### Task

A file user_purchases.csv contains transaction data of users with a weekly subscription.
Each row represents one transaction.

Columns:
- transaction_id — unique ID of each transaction
- original_transaction_id — ID of the first transaction of a user (the same for all transactions of this user)
- purchase_date — date of the transaction
- original_purchase_date — date of the first transaction of the subscription

Conditions:
- Subscription length is 7 days
- The first week is a free trial
- Each subsequent week costs $4.99

Metrics to calculate:
- Trial-to-paid conversion rate — the share of users who made at least one paid transaction after the 7-day trial ended.
- Average net revenue per user at day 14 — the average total revenue per user within the first 14 days from the start of the trial.
- Forecasted average net revenue per user at day 90 — the expected average total revenue per user by day 90.

## Data Analysis Plan

### 1. Data understanding
- Load CSV file
- Inspect schema (columns, dtypes, row count)
- Understand granularity (1 row = 1 transaction)
- Identify primary keys (`transaction_id`) and user keys (`original_transaction_id`)
- Understand the meaning of each column and its role in metrics

### 2. Data quality checks
- Missing values
- Duplicates
- Unique `transaction_id`
- Consistent `original_purchase_date` per user
- Logical date order (`purchase_date` ≥ `original_purchase_date`)
- Increasing `purchase_date` within each user
- One transaction per user per 7-day period (no `gap_days` < 7)
- Reasonable `days_since_start` values (0, ~7, ~14, …)

### 3. Feature engineering
- Calculate `days_since_start` (days from trial start to each transaction)
- Flag trial vs paid transactions (`is_paid`)
- Assign subscription week number (`week_number`)

### 4. Metric calculations
- Trial-to-paid conversion rate
- Average net revenue per user at day 14
- Forecasted average net revenue per user at day 90


In [2]:
import pandas as pd

# --- Load data ---
df = pd.read_csv("user_purchases.csv", sep=';')

# --- Basic info ---
print("Shape (rows, columns):", df.shape)
print("\nColumn names:", df.columns.tolist())
print("\nData types:")
print(df.dtypes)

# --- First 5 rows ---
print("\nSample rows:")
print(df.head())

Shape (rows, columns): (5000, 4)

Column names: ['original_transaction_id', 'original_purchase_date', 'transaction_id', 'purchase_date']

Data types:
original_transaction_id     int64
original_purchase_date     object
transaction_id              int64
purchase_date              object
dtype: object

Sample rows:
   original_transaction_id original_purchase_date   transaction_id  \
0          100001666350205             2024-05-01  100001673857680   
1          100001666350205             2024-05-01  100001666350205   
2          100001666350205             2024-05-01  100001683102502   
3          100001666928347             2024-05-01  100001666928347   
4          100001667091726             2024-05-01  100001667091726   

  purchase_date  
0    2024-05-08  
1    2024-05-01  
2    2024-05-23  
3    2024-05-01  
4    2024-05-01  


In [3]:
# --- Convert date columns to datetime ---
df["original_purchase_date"] = pd.to_datetime(df["original_purchase_date"], errors="coerce")
df["purchase_date"] = pd.to_datetime(df["purchase_date"], errors="coerce")

# check results
print(df.dtypes.head())


original_transaction_id             int64
original_purchase_date     datetime64[ns]
transaction_id                      int64
purchase_date              datetime64[ns]
dtype: object


In [4]:
# --- Check missing values ---
print("Missing values per column:")
print(df.isna().sum())

# --- Check duplicates ---
print("\nNumber of duplicate rows:", df.duplicated().sum())

# --- Check uniqueness of transaction_id ---
print("\nUnique transaction_id:", df["transaction_id"].nunique())
print("Total rows:", len(df))

# --- Check original_purchase_date consistency ---
user_dates = df.groupby("original_transaction_id")["original_purchase_date"].nunique()
print("\nUsers with more than one original_purchase_date:", (user_dates > 1).sum())

# --- Check date order ---
invalid_dates = df[df["purchase_date"] < df["original_purchase_date"]]
print("\nRows where purchase_date < original_purchase_date:", len(invalid_dates))


Missing values per column:
original_transaction_id    0
original_purchase_date     0
transaction_id             0
purchase_date              0
dtype: int64

Number of duplicate rows: 0

Unique transaction_id: 5000
Total rows: 5000

Users with more than one original_purchase_date: 0

Rows where purchase_date < original_purchase_date: 1


In [5]:
# remove rows where purchase_date < original_purchase_date
df = df[df["purchase_date"] >= df["original_purchase_date"]]

print("Rows after cleaning:", len(df))


Rows after cleaning: 4999


In [6]:
# group by user and calculate day gaps between consecutive transactions
df_sorted = df.sort_values(["original_transaction_id", "purchase_date"])

# for each user: difference in days between this and previous transaction
df_sorted["gap_days"] = df_sorted.groupby("original_transaction_id")["purchase_date"].diff().dt.days

# find suspicious cases (<7 days)
too_frequent = df_sorted[df_sorted["gap_days"].notna() & (df_sorted["gap_days"] < 7)]

print("Users who paid more than once within 7 days:", too_frequent["original_transaction_id"].nunique())
print("Total suspicious transactions:", len(too_frequent))

# optional: show first few suspicious rows
print("\nExamples:")
print(too_frequent.head())


Users who paid more than once within 7 days: 5
Total suspicious transactions: 6

Examples:
      original_transaction_id original_purchase_date    transaction_id  \
1253          120002258196905             2024-05-05  1120000010058780   
1252          120002258196905             2024-05-05  1120000011992537   
3235          160001953881179             2024-05-06   160001976858363   
4026          180002056946662             2024-05-04   180002098345945   
890          1110000009798992             2024-05-30   110002150393712   

     purchase_date  gap_days  
1253    2024-05-11       6.0  
1252    2024-05-13       2.0  
3235    2024-05-22       0.0  
4026    2024-06-02       2.0  
890     2024-06-05       6.0  


In [7]:
# remove suspicious transactions with gap_days < 7
clean_df = df_sorted[
    (df_sorted["gap_days"].isna()) | (df_sorted["gap_days"] >= 7)
].copy()

print("Rows before:", len(df_sorted))
print("Rows after cleaning:", len(clean_df))


Rows before: 4999
Rows after cleaning: 4993


In [8]:
# calculate days_since_start
clean_df["days_since_start"] = (
    clean_df["purchase_date"] - clean_df["original_purchase_date"]
).dt.days

# mark first transaction per user as trial, others as paid
clean_df["is_paid"] = (
    clean_df.groupby("original_transaction_id")["purchase_date"]
    .rank(method="first")
    > 1
)

# check result
print(clean_df[["original_transaction_id","purchase_date","days_since_start","is_paid"]].head(10))


    original_transaction_id purchase_date  days_since_start  is_paid
1           100001666350205    2024-05-01                 0    False
0           100001666350205    2024-05-08                 7     True
2           100001666350205    2024-05-23                22     True
3           100001666928347    2024-05-01                 0    False
4           100001667091726    2024-05-01                 0    False
10          100001667348375    2024-05-01                 0    False
7           100001667348375    2024-05-08                 7     True
5           100001667348375    2024-05-16                15     True
8           100001667348375    2024-05-23                22     True
9           100001667348375    2024-05-30                29     True


In [9]:
# each 7-day block is one subscription week
clean_df["week_number"] = (clean_df["days_since_start"] // 7).astype(int)

# check result
print(clean_df[["original_transaction_id","purchase_date","days_since_start","week_number","is_paid"]].head(10))


    original_transaction_id purchase_date  days_since_start  week_number  \
1           100001666350205    2024-05-01                 0            0   
0           100001666350205    2024-05-08                 7            1   
2           100001666350205    2024-05-23                22            3   
3           100001666928347    2024-05-01                 0            0   
4           100001667091726    2024-05-01                 0            0   
10          100001667348375    2024-05-01                 0            0   
7           100001667348375    2024-05-08                 7            1   
5           100001667348375    2024-05-16                15            2   
8           100001667348375    2024-05-23                22            3   
9           100001667348375    2024-05-30                29            4   

    is_paid  
1     False  
0      True  
2      True  
3     False  
4     False  
10    False  
7      True  
5      True  
8      True  
9      True  


In [10]:
# count total users
total_users = clean_df["original_transaction_id"].nunique()

# find users who have at least one paid transaction
paid_users = (
    clean_df[clean_df["is_paid"] == True]["original_transaction_id"]
    .nunique()
)

# calculate conversion
conversion_rate = paid_users / total_users

# print results
print("Total users:", total_users)
print("Users with at least one paid transaction:", paid_users)
print("Trial-to-paid conversion rate:", round(conversion_rate * 100, 2), "%")


Total users: 3129
Users with at least one paid transaction: 1019
Trial-to-paid conversion rate: 32.57 %


In [11]:
# 4.1 Trial-to-paid conversion
total_users = clean_df["original_transaction_id"].nunique()
paid_users = clean_df.loc[clean_df["is_paid"], "original_transaction_id"].nunique()
conversion_rate = paid_users / total_users
print("Conversion rate:", round(conversion_rate * 100, 2), "%")

# 4.2 Avg net revenue per user at day 14 (включительно)
PRICE = 4.99
paid_before_14 = clean_df[(clean_df["is_paid"]) & (clean_df["days_since_start"] <= 14)]
total_rev_14 = len(paid_before_14) * PRICE
avg_rev_14 = total_rev_14 / total_users
print("Average revenue per user at day 14: $", round(avg_rev_14, 2))


Conversion rate: 32.57 %
Average revenue per user at day 14: $ 2.21


In [25]:
PRICE = 4.99

# total number of users
n_users = df["original_transaction_id"].nunique()

# conversion rate: users with at least one transaction after trial (>= 7 days)
converted_users = df[df["days_since_start"] >= 7]["original_transaction_id"].nunique()
conversion_rate = converted_users / n_users

# business assumption: converted users make on average 7.3 paid weeks within 90 days
weeks_per_paying_user = 7.3

# forecasted ARPU_90
arpu_90 = conversion_rate * weeks_per_paying_user * PRICE

print("Conversion rate:", round(conversion_rate * 100, 2), "%")
print("Forecasted ARPU_90 (predicted): $", round(arpu_90, 2))


Conversion rate: 32.73 %
Forecasted ARPU_90 (predicted): $ 11.92


## Results

- **Trial-to-paid conversion rate:** 32.73%  
  → about one third of users made at least one paid transaction after the 7-day trial

- **Average net revenue per user at day 14:** $2.21  
  → low early monetization due to limited number of paid transactions within the first 14 days

- **Forecasted average net revenue per user at day 90:** $11.92  
  → based on the assumption that each converted user makes 7.3 paid weeks within 90 days
